In [ ]:
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request

In [4]:
# REFERENCE: https://github.com/BMMR/pythonBolt/blob/54428902c54fd8b01f7fc3e00a39145a72413da3/google_apis.py
def create_service(client_secret_file, api_name, api_version, *scopes, prefix=''):
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]

    creds = None
    working_dir = os.getcwd()
    token_dir = 'token files'
    token_file = f'token_{API_SERVICE_NAME}_{API_VERSION}{prefix}.json'

    ### Check if token dir exists first, if not, create the folder
    if not os.path.exists(os.path.join(working_dir, token_dir)):
        os.mkdir(os.path.join(working_dir, token_dir))

    if os.path.exists(os.path.join(working_dir, token_dir, token_file)):
        creds = Credentials.from_authorized_user_file(os.path.join(working_dir, token_dir, token_file), SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open(os.path.join(working_dir, token_dir, token_file), 'w') as token:
            token.write(creds.to_json())

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=creds, static_discovery=False)
        print(API_SERVICE_NAME, API_VERSION, 'service created successfully')
        return service
    except Exception as e:
        print(e)
        print(f'Failed to create service instance for {API_SERVICE_NAME}')
        os.remove(os.path.join(working_dir, token_dir, token_file))
        return None

In [2]:
client_secret_file = 'client_secret.json'
API_SERVICE_NAME = 'drive'
API_VERSION = 'v3'
SCOPES =['https://www.googleapis.com/auth/drive']

In [5]:
service = create_service(client_secret_file, API_SERVICE_NAME, API_VERSION, SCOPES)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=478639231328-tntgmc6q72bo1v5psvhsfd0dmd9sf9lp.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49687%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=JciP0MSq5kFKMscl2o4OiTienr0x1T&access_type=offline
drive v3 service created successfully


In [16]:
folder_id = '1EtI0nZW1DenKKC4-P1WX8PQHfubRJBWE'
query= f"parents = '{folder_id}'"

In [26]:
directories = ['recordings', 'transcripts', 'feedback']
for f in directories:
    file_metadata = {
        'name': f,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [f'{folder_id}']
    }
    service.files().create(body=file_metadata, supportsAllDrives=True, fields='id').execute()

In [14]:
service.files().list(supportsAllDrives=True, includeItemsFromAllDrives=True, q=query).execute()

{'kind': 'drive#fileList',
 'incompleteSearch': False,
 'files': [{'kind': 'drive#file',
   'driveId': '0AGWDQtrixTcNUk9PVA',
   'mimeType': 'application/vnd.google-apps.folder',
   'id': '1knH9HXDnDYvTomDcVG6lFy2Bfp6qSRyu',
   'name': 'Test',
   'teamDriveId': '0AGWDQtrixTcNUk9PVA'}]}